### Ensuring Feature Consistency Between Training & InferencePipelines:

**Task 1**: Consistent Feature Preparation
- Step 1: Write a function for data preprocessing and imputation shared by both training and inference pipelines.
- Step 2: Demonstrate consistent application on both datasets.

In [ ]:
# write your code from here

**Task 2**: Pipeline Integration
- Step 1: Use sklearn pipelines to encapsulate the preprocessing steps.
- Step 2: Configure identical pipelines for both training and building inference models.

In [ ]:
# write your code from here

**Task 3**: Saving and Loading Preprocessing Models
- Step 1: Save the transformation model after fitting it to the training data.
- Step 2: Load and apply the saved model during inference.

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import joblib
import os

def load_data_with_missing():
    """
    Load Boston housing dataset and inject missing values in 'CRIM' column.
    Returns a DataFrame.
    """
    try:
        boston = load_boston()
        df = pd.DataFrame(boston.data, columns=boston.feature_names)
        df.loc[::10, 'CRIM'] = np.nan
        return df
    except Exception as e:
        raise RuntimeError(f"Failed to load or prepare dataset: {e}")

def validate_dataframe(df):
    """
    Validate input DataFrame:
    - Must be a DataFrame
    - Must contain only numeric columns (for this pipeline)
    Raises ValueError if invalid.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input is not a pandas DataFrame.")
    if not all(np.issubdtype(dtype, np.number) for dtype in df.dtypes):
        raise ValueError("DataFrame contains non-numeric columns, which are unsupported.")

def create_preprocessing_pipeline():
    """
    Create a sklearn Pipeline for imputation and scaling.
    """
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    return pipeline

def preprocess_data(df, pipeline=None, fit_pipeline=True):
    """
    Preprocess input DataFrame using pipeline.
    If pipeline is None, create a new one.
    If fit_pipeline=True, fit and transform data.
    Otherwise, only transform.
    Returns tuple: (processed numpy array, pipeline used)
    """
    validate_dataframe(df)
    if pipeline is None:
        pipeline = create_preprocessing_pipeline()
    try:
        if fit_pipeline:
            processed = pipeline.fit_transform(df)
        else:
            processed = pipeline.transform(df)
    except Exception as e:
        raise RuntimeError(f"Error during preprocessing: {e}")
    return processed, pipeline

def save_pipeline(pipeline, filename):
    """
    Save sklearn pipeline to disk using joblib with error handling.
    """
    try:
        joblib.dump(pipeline, filename)
    except Exception as e:
        raise IOError(f"Failed to save pipeline to {filename}: {e}")

def load_pipeline(filename):
    """
    Load sklearn pipeline from disk with error handling.
    """
    if not os.path.exists(filename):
        raise FileNotFoundError(f"Pipeline file {filename} does not exist.")
    try:
        pipeline = joblib.load(filename)
        return pipeline
    except Exception as e:
        raise IOError(f"Failed to load pipeline from {filename}: {e}")

# === Example usage ===
if __name__ == "__main__":
    # Load and prepare data
    df = load_data_with_missing()

    # Split train/test
    X_train, X_test = train_test_split(df, test_size=0.3, random_state=42)

    # Preprocess training data (fit pipeline)
    X_train_processed, pipeline = preprocess_data(X_train, fit_pipeline=True)
    print("Training data processed shape:", X_train_processed.shape)

    # Preprocess test data (transform only)
    X_test_processed, _ = preprocess_data(X_test, pipeline=pipeline, fit_pipeline=False)
    print("Inference data processed shape:", X_test_processed.shape)

    # Save the fitted pipeline
    pipeline_file = "preprocessing_pipeline.joblib"
    save_pipeline(pipeline, pipeline_file)
    print(f"Pipeline saved to {pipeline_file}")

    # Load pipeline and preprocess inference data again
    loaded_pipeline = load_pipeline(pipeline_file)
    X_test_processed_loaded = loaded_pipeline.transform(X_test)
    print("Loaded pipeline processed inference data shape:", X_test_processed_loaded.shape)

# === Basic unit tests ===
import unittest

class TestPreprocessing(unittest.TestCase):
    def test_validate_dataframe_raises(self):
        with self.assertRaises(ValueError):
            validate_dataframe("not a dataframe")
        with self.assertRaises(ValueError):
            validate_dataframe(pd.DataFrame({'A': ['a','b']}))  # non-numeric

    def test_imputation_pipeline(self):
        df_test = pd.DataFrame({'A': [1, 2, np.nan, 4]})
        processed, pipe = preprocess_data(df_test)
        self.assertFalse(np.isnan(processed).any())
        # Check that mean imputation replaced nan with mean 2.3333
        self.assertAlmostEqual(processed[2,0], (1+2+4)/3, places=4)

    def test_save_load_pipeline(self):
        pipeline = create_preprocessing_pipeline()
        filename = "test_pipeline.joblib"
        save_pipeline(pipeline, filename)
        self.assertTrue(os.path.exists(filename))
        loaded_pipe = load_pipeline(filename)
        self.assertIsNotNone(loaded_pipe)
        os.remove(filename)

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

In [1]:
# write your code from here
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import joblib  # For saving/loading pipeline

# --- Task 1: Consistent Feature Preparation ---

def load_data_with_missing():
    """Load Boston housing dataset and artificially inject missing values."""
    boston = load_boston()
    df = pd.DataFrame(boston.data, columns=boston.feature_names)
    # Inject missing values in 'CRIM' column (every 10th row)
    df.loc[::10, 'CRIM'] = np.nan
    return df

def preprocess_data(df, pipeline=None, fit_pipeline=True):
    """
    Preprocess dataset using given pipeline.
    If no pipeline provided, create a new one (SimpleImputer + StandardScaler).
    If fit_pipeline=True, fit and transform; else only transform.
    """
    if pipeline is None:
        pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ])
    if fit_pipeline:
        processed = pipeline.fit_transform(df)
    else:
        processed = pipeline.transform(df)
    return processed, pipeline

# --- Demonstrate consistent application on train & inference ---

# Load full dataset and split into train/test to simulate inference data
df = load_data_with_missing()
X_train, X_test = train_test_split(df, test_size=0.3, random_state=42)

# Preprocess training data (fit pipeline here)
X_train_processed, pipeline = preprocess_data(X_train, fit_pipeline=True)

# Preprocess inference data (use same pipeline, no fitting)
X_test_processed, _ = preprocess_data(X_test, pipeline=pipeline, fit_pipeline=False)

print("Training data processed shape:", X_train_processed.shape)
print("Inference data processed shape:", X_test_processed.shape)


# --- Task 2: Pipeline Integration ---
# Already done above by creating and using sklearn Pipeline
# This ensures identical steps applied during train and inference

# --- Task 3: Saving and Loading Preprocessing Models ---

# Save the fitted pipeline to disk
pipeline_filename = 'preprocessing_pipeline.joblib'
joblib.dump(pipeline, pipeline_filename)
print(f"\nPipeline saved to {pipeline_filename}")

# Load the pipeline from disk and apply to new data (simulate inference)
loaded_pipeline = joblib.load(pipeline_filename)
X_new_processed = loaded_pipeline.transform(X_test)

print("Loaded pipeline processed inference data shape:", X_new_processed.shape)


ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[: